In [1]:
import pandas as pd
import numpy as np
from scipy import sparse
%matplotlib inline
import matplotlib.pyplot as plt
import mglearn
import sklearn
from sklearn.model_selection import train_test_split

from matplotlib import rc
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

머신 러닝에서는, 

데이터가 어떤 형태의 feature로 구현되어 있는가(continuous / categorical (discrete))보다는

'데이터를 어떻게 표현하는가'가 모델 성능에 영향을 미친다. 예컨대 2, 3장에서 본 scale 조정이 모델 성능에 긍정적인 영향을 미친다.

또는 feature의 상호작용 값(곱셉)이나 제곱값, 일반적인 다항식을 특성으로 넣어서 성능 향상도 가능하다.

#### 이처럼 특정 애플리케이션에 가장 적합한 데이터 표현을 찾는 걸 feature engineering이라고 한다.

올바른 데이터 표현은 적절한 hyperparameter를 선택하는 것보다 성능에 더 큰 영향을 미친다.



In [2]:
import os
data = pd.read_csv(os.path.join(mglearn.datasets.DATA_PATH, "adult.data"),header=None,
                  index_col = False, names = ['age','workclass','fnlwgt','education','education_num',
                                             'marital_status','occupation','relationship','race','gender',
                                             'capital_gain','capital_loss','hours_per_week','native_country',
                                             'income'])
data = data[['age','workclass','education','gender','hours_per_week','occupation','income']]
data.head()

,age,workclass,education,gender,hours_per_week,occupation,income
0,39,State-gov,Bachelors,Male,40,Adm-clerical,<=50K
1,50,Self-emp-not-inc,Bachelors,Male,13,Exec-managerial,<=50K
2,38,Private,HS-grad,Male,40,Handlers-cleaners,<=50K
3,53,Private,11th,Male,40,Handlers-cleaners,<=50K
4,28,Private,Bachelors,Female,40,Prof-specialty,<=50K


1. column에 어떤 범주형 데이터(categorical)가 있는지 확인한다.

사용자로부터 입력받은 값일 경우 오탈자 / 범주 밖의 값이 있을 수 있다. value_counts()로 확인

2. get_dummies 함수로 categorical date encoding

연속형 특성(age, hours_per_week)은 그대로, categorical feature는 값마다 새로운 feature로 확장되었음.

In [4]:
print('original features: {}'.format(data.columns),'\n')
data_dummies = pd.get_dummies(data)
print('get_dummies features: \n', list(data_dummies.columns))

original features: Index(['age', 'workclass', 'education', 'gender', 'hours_per_week',
       'occupation', 'income'],
      dtype='object') 

get_dummies features: 
 ['age', 'hours_per_week', 'workclass_ ?', 'workclass_ Federal-gov', 'workclass_ Local-gov', 'workclass_ Never-worked', 'workclass_ Private', 'workclass_ Self-emp-inc', 'workclass_ Self-emp-not-inc', 'workclass_ State-gov', 'workclass_ Without-pay', 'education_ 10th', 'education_ 11th', 'education_ 12th', 'education_ 1st-4th', 'education_ 5th-6th', 'education_ 7th-8th', 'education_ 9th', 'education_ Assoc-acdm', 'education_ Assoc-voc', 'education_ Bachelors', 'education_ Doctorate', 'education_ HS-grad', 'education_ Masters', 'education_ Preschool', 'education_ Prof-school', 'education_ Some-college', 'gender_ Female', 'gender_ Male', 'occupation_ ?', 'occupation_ Adm-clerical', 'occupation_ Armed-Forces', 'occupation_ Craft-repair', 'occupation_ Exec-managerial', 'occupation_ Farming-fishing', 'occupation_ Handlers-cleane

data_dummmies의 values로 Numpy 배열 변환이 가능함. 이걸로 모델 학습.

모델 학습시키기 이전, 데이터로부터 '타겟값'을 분리해야 함. 이 경우 income_ <=50k, income_ >50k.

출력값 또는 출력값에서 유도된 변수를 feature에 넣지 않도록 유의.

여기서는 특성을 포함한, age부터 occupation_ Transport-moving까지를 쓸 거다.

In [7]:
features = data_dummies.loc[:,'age':'occupation_ Transport-moving']
#Numpy 배열 추출
X = features.values
y = data_dummies['income_ >50K'].values

훈련 데이터와 테스트 데이터의 feature 속성 / 이름이 다를 경우 모델 예측성능이 좋지 않다.

특히 get_dummies로 범주화한 데이터의 경우 column이름이나 위치가 달라질 수 있다.

따라서 train / test 데이터가 모두 포함된 dataframe을 train / test로 나누는 방식이나

train, test 데이터를 각각 써야 한다면 면밀히 비교해야 한다.

In [ ]:
# pandas의 get_dummies 함수는, 숫자는 전부 연속형이라 판단해 feature를 만들지 않는다.
# 어떤 column이 categorical 변수인지를 지정할 수 있는 게 OneHotEncoder